This code is used for converting videos to images which are helpful in training CycleGAN and generating TSNEs

Importing Libraries

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, ConcatDataset
from torchvision import transforms, models
import torch.optim as optim
import os
import random
import cv2
import numpy as np

Labels

In [ ]:
class_label_real = 0
class_label_attack = 1

Dataset Path

In [ ]:
data_path_videos = '/dataset/videos'

Function for Loading Dataset

In [ ]:
def load_samples(path, class_label, transform, save_path, counter_dict):
    frames = read_all_frames(path)
    total_frames = list(range(0, frames.shape[0], 1)) # Read all frames of video
    selected_samples = random.sample(total_frames, 1) # Extract one randomly frame from each video. Change 1 to any desired frame whatever needed for extraction. For Cycle-GAN training we need more frames but for TSNE 1 frame is enough.
    samples = []

    for sample_idx in selected_samples:
        frame = transform(frames[sample_idx].squeeze())
        label = class_label

        # Get the counter for the current class
        counter = counter_dict.get(class_label, 0)

        # Save the frame to disk with a unique filename
        save_filename = f"{class_label}_{counter}_{sample_idx}.png"
        save_filepath = os.path.join(save_path, save_filename)
        cv2.imwrite(save_filepath, frame.numpy().transpose(1, 2, 0) * 255)

        # Update the counter for the current class
        counter_dict[class_label] = counter + 1

        samples.append((frame, label))

    return samples

def read_all_frames(video_path): # reads all frames from a particular video and converts them to PyTorch tensors.
    frame_list = []
    video = cv2.VideoCapture(video_path)
    success = True
    while success:
        success, frame = video.read()
        if success:
            frame = cv2.resize(frame, (256, 256), interpolation=cv2.INTER_AREA) #framesize kept 256 x 256
            frame_list.append(frame)
    frame_list = np.array(frame_list)
    return frame_list

class VideoDataset(Dataset):
    def __init__(self, data_path, class_label, save_path, counter_dict):
        self.data_path = data_path #path for directory containing video files
        self.video_files = [file for file in os.listdir(data_path) if file.endswith('.avi')] #list of video files in the specified directory #.mov for RA and RM, .avi for OULU-NPU, .mp4 for RY
        self.class_label = class_label #manually assign class_label for your desired class while loading
        self.data_length = len(self.video_files) 
        self.transform = transforms.Compose([transforms.ToTensor()])
        self.save_path = save_path # Added save_path parameter
        self.counter_dict = counter_dict

    def __len__(self): # returns the total number of samples in the dataset
        return self.data_length

    def __getitem__(self, idx): # loads and returns a sample from the dataset at the given index
        file = self.video_files[idx]
        path = os.path.join(self.data_path, file)
        frames= load_samples(path, self.class_label, self.transform, self.save_path, self.counter_dict) 

        return frames

Loading Dataset Videos, Assigning Label, & Images Path  

In [ ]:
counter_dict = {}
save_path_images = '/folder/images'
save_dataset = VideoDataset(data_path_videos, class_label, save_path_images, counter_dict) # Assign class_label_real = 0 or class_label_attack = 1

Extracting Images to Desired Save Path

In [ ]:
all_samples = []
for idx in range(len(save_dataset)):
    samples = save_dataset[idx]
    all_samples.extend(samples)

Check Total No. of Saved Images

In [ ]:
print(len(os.listdir(save_path_images)))